<a href="https://colab.research.google.com/github/DongWooLeee/Tobigs19/blob/main/Vision_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm import tqdm

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms


# 코드 다시 돌리기 위한 seed 고정
import random
import numpy as np
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:
def conv_layer(in_dim, out_dim, pool=False):
  kernel_size=3
  layers = [
      nn.Conv2d(in_dim, out_dim, kernel_size = kernel_size, padding=1),
      nn.BatchNorm2d(out_dim),
      nn.ReLU()]
  if pool:#pooling의 경우 
    layers.append(nn.MaxPool2d(2))
  return nn.Sequential(*layers)


In [16]:
class MyResNet(nn.Module):
    def __init__(self, in_dim, num_classes):
        super().__init__()
        
        self.conv1 = conv_layer(in_dim, 64)
        self.conv2 = conv_layer(64, 128, pool=True) 
        self.res1 = nn.Sequential(conv_layer(128, 128), conv_layer(128, 128)) #Residual Block1(X)
        
        self.conv3 = conv_layer(128, 256, pool=True)
        self.conv4 = conv_layer(256, 512, pool=True) 
        self.res2 = nn.Sequential(conv_layer(512, 512), conv_layer(512, 512)) #Residual Block(X)
        self.conv5 = conv_layer(512, 1028, pool=True)
        self.res3 = nn.Sequential(conv_layer(1028, 1028), conv_layer(1028, 1028))  #Residual Block(X)
        
        self.fc = nn.Sequential(nn.MaxPool2d(2), # 1028 x 1 x 1
                                        nn.Flatten(), # 1028 
                                        nn.Linear(1028, num_classes)) # 1028 -> 100
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.conv5(out)
        out = self.res3(out) + out
        out = self.fc(out)
        return out

In [17]:
model = MyResNet(3, 100).to(device)
model

MyResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (conv3): Sequential(
    (0): C

In [18]:
train_transform = transforms.Compose([    
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),(0.2675, 0.2565, 0.2761))
])        

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408),(0.2675, 0.2565, 0.2761))
])    

train = torchvision.datasets.CIFAR100(root="./", train=True, download=True, transform=train_transform)
test = torchvision.datasets.CIFAR100(root="./", train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=256,
                                           shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=256,
                                          shuffle=False, num_workers=2)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

Files already downloaded and verified
Files already downloaded and verified


In [20]:
for epoch in range(20):
    model.train()
    running_loss = 0.0
    best_acc = 0.0
    best_model_wts = model.state_dict()
    print(f"train epoch: {epoch+1}----------------")
    for img, label in tqdm(train_loader):
        img = img.to(device)
        label = label.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    correct, all_data = 0,0
    print("\ntrain_loss : ", running_loss / len(train_loader))
    model.eval()
    for img, label in test_loader:
        with torch.no_grad():
            img = img.to(device)
            label = label.to(device)
            output = model(img)

            correct += torch.sum(torch.argmax(output, dim=1) == label).item()
            all_data += len(label)
    print("val_acc : ", correct / all_data)
    if correct / all_data > best_acc:
      best_acc = correct / all_data
      best_model_wts = model.state_dict()

train epoch: 1----------------


100%|██████████| 196/196 [00:36<00:00,  5.30it/s]


train_loss :  0.02250393445613053


val_acc :  0.6205
train epoch: 2----------------


100%|██████████| 196/196 [00:37<00:00,  5.26it/s]

...


train_loss :  0.002675321548868313


val_acc :  0.6393
train epoch: 20----------------


100%|██████████| 196/196 [00:37<00:00,  5.28it/s]


train_loss :  0.002200939578962826


val_acc :  0.639
